# Conversational agent

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.tools import tool

In [28]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class NewsAPI(BaseModel):
    keyword: str = Field(..., description="keyword to search for in the news articles")

@tool(args_schema=NewsAPI)
def get_keyword_news(keyword: str) -> dict:
    """Fetch news articles for the given specific topic or keyword"""
    
    API_KEY = "18d86d14e08b493bbe47f16380c4ad08"
    BASE_URL = f"https://newsapi.org/v2/everything"
    
    # Parameters for the request
    params = {
        'q': keyword,
        'pageSize': 5,
        'apiKey': API_KEY,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")
    
    return results

In [29]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [30]:
tools = [get_keyword_news, search_wikipedia]

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [32]:
functions = [format_tool_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0).bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [35]:
result = chain.invoke({"input": "what is news about Mars"})
result

AgentActionMessageLog(tool='get_keyword_news', tool_input={'keyword': 'Mars'}, log="\nInvoking: `get_keyword_news` with `{'keyword': 'Mars'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_keyword_news', 'arguments': '{"keyword":"Mars"}'}})])

In [34]:
result.tool

'get_keyword_news'

In [10]:
result.tool_input

{'keyword': 'Mars'}

In [36]:
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [37]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [38]:
result1 = chain.invoke({
    "input": "what is news on Mars?",
    "agent_scratchpad": []
})

In [39]:
result1

AgentActionMessageLog(tool='get_keyword_news', tool_input={'keyword': 'Mars'}, log="\nInvoking: `get_keyword_news` with `{'keyword': 'Mars'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_keyword_news', 'arguments': '{"keyword":"Mars"}'}})])

In [40]:
observation = get_keyword_news(result1.tool_input)

In [41]:
observation

{'status': 'ok',
 'totalResults': 7464,
 'articles': [{'source': {'id': 'wired', 'name': 'Wired'},
   'author': 'Becky Ferreira',
   'title': 'To Find Alien Life, We Might Have to Kill It',
   'description': 'Missions to explore other worlds, like Mars or Saturn’s moon Titan, could disrupt or destroy extraterrestrial life in the process of seeking it.',
   'url': 'https://www.wired.com/story/alien-life-killing-ethics-mars-rovers-titan-moon-europa-clipper-dragonfly/',
   'urlToImage': 'https://media.wired.com/photos/66984a1d48990fd300c9e57c/191:100/w_1280,c_limit/science_ethics_killing_alien_life.jpg',
   'publishedAt': '2024-07-18T11:00:00Z',
   'content': 'When is it OK to kill an alien life-form?\r\nIn the movies, the answer is usually pretty simple: Its OK in self-defense, especially if it inspires a rousing speech about human exceptionalism. But in th… [+3034 chars]'},
  {'source': {'id': None, 'name': 'Gizmodo.com'},
   'author': 'Adam Kovac',
   'title': 'Mars Missions Face Unexp

In [20]:
type(result1)

langchain.schema.agent.AgentActionMessageLog

In [42]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [43]:
result1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_keyword_news', 'arguments': '{"keyword":"Mars"}'}})]

In [44]:
format_to_openai_functions([(result1, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_keyword_news', 'arguments': '{"keyword":"Mars"}'}}),
 FunctionMessage(content='{"status": "ok", "totalResults": 7464, "articles": [{"source": {"id": "wired", "name": "Wired"}, "author": "Becky Ferreira", "title": "To Find Alien Life, We Might Have to Kill It", "description": "Missions to explore other worlds, like Mars or Saturn’s moon Titan, could disrupt or destroy extraterrestrial life in the process of seeking it.", "url": "https://www.wired.com/story/alien-life-killing-ethics-mars-rovers-titan-moon-europa-clipper-dragonfly/", "urlToImage": "https://media.wired.com/photos/66984a1d48990fd300c9e57c/191:100/w_1280,c_limit/science_ethics_killing_alien_life.jpg", "publishedAt": "2024-07-18T11:00:00Z", "content": "When is it OK to kill an alien life-form?\\r\\nIn the movies, the answer is usually pretty simple: Its OK in self-defense, especially if it inspires a rousing speech about human exceptionalism. But in th… 

In [49]:
result2 = chain.invoke({
    "input": "what is news about moon?", 
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})

In [50]:
result2

AgentFinish(return_values={'output': "I found some interesting news articles related to Mars:\n\n1. Title: [To Find Alien Life, We Might Have to Kill It](https://www.wired.com/story/alien-life-killing-ethics-mars-rovers-titan-moon-europa-clipper-dragonfly/)\n   Source: Wired\n   Description: Missions to explore other worlds, like Mars or Saturn’s moon Titan, could disrupt or destroy extraterrestrial life in the process of seeking it.\n   Published At: 2024-07-18\n\n2. Title: [Mars Missions Face Unexpected Challenge: Expired Meds](https://gizmodo.com/mars-missions-face-unexpected-challenge-expired-meds-2000478111)\n   Source: Gizmodo.com\n   Description: Astronauts on Mars missions may face challenges with expired medications.\n   Published At: 2024-07-24\n\n3. Title: [Elon Musk Needs a Win, Says He Wants to Fight Zuck Again](https://gizmodo.com/elon-musk-needs-a-win-says-he-wants-to-fight-zuck-again-2000479042)\n   Source: Gizmodo.com\n   Description: Elon Musk expresses interest in fi

In [52]:
from langchain.schema.agent import AgentFinish
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input, 
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_keyword_news": get_keyword_news,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [53]:
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [54]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input, 
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_keyword_news": get_keyword_news,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [55]:
run_agent("what is news about moon?")

AgentFinish(return_values={'output': "I found some news articles about the moon:\n\n1. **[Newly Discovered Moon Caves Could One Day House Astronauts](https://www.wired.com/story/moon-tunnels-caves-astronauts-moon-colony/)** - Analysis of lunar imagery has ended a longstanding debate over whether there are accessible underground areas on the Moon; an emptied lava tube in the Sea of Tranquility is of particular interest. (Published on: 2024-07-18)\n\n2. **[To Find Alien Life, We Might Have to Kill It](https://www.wired.com/story/alien-life-killing-ethics-mars-rovers-titan-moon-europa-clipper-dragonfly/)** - Missions to explore other worlds, like Mars or Saturn’s moon Titan, could disrupt or destroy extraterrestrial life in the process of seeking it. (Published on: 2024-07-18)\n\n3. **[Trump's Crypto Embrace Could Be a Disaster for Bitcoin](https://www.wired.com/story/donald-trump-bitcoin-reserve-promises/)** - At the Bitcoin 2024 conference in Nashville, Donald Trump promised the crypto 

In [56]:
run_agent("what is langchain?")

AgentFinish(return_values={'output': 'LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). It is a language model integration framework with use-cases including document analysis and summarization, chatbots, and code analysis.'}, log='LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). It is a language model integration framework with use-cases including document analysis and summarization, chatbots, and code analysis.')

In [57]:
run_agent("hi!")

AgentFinish(return_values={'output': 'Well, hello there! How can I assist you today?'}, log='Well, hello there! How can I assist you today?')

In [58]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [59]:
agent_executor.invoke({"input": "what is langchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Langchain'}`


Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Retrieval-augmented generation
Summary: Retrieval augmented generation (RAG) is a type of information retrieval process. It modifies interactions with a large language model (LLM) so that it responds to queries with reference to a specified set of documents, using it in preference to information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  
Use cases include providing chatbot access to internal company data, or giving factual information only from an authoritative source.

Page:

{'input': 'what is langchain?',
 'output': 'LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). It is a language model integration framework with use-cases including document analysis and summarization, chatbots, and code analysis.'}

In [60]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'output': 'Nice to meet you, Bob! How can I assist you today?'}

In [61]:
agent_executor.invoke({"input": "what is my name"})



> Entering new AgentExecutor chain...
I'm sorry, I don't have access to your personal information like your name. How can I assist you today?

> Finished chain.


{'input': 'what is my name',
 'output': "I'm sorry, I don't have access to your personal information like your name. How can I assist you today?"}

In [62]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [63]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [64]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [65]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [66]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?')],
 'output': 'Nice to meet you, Bob! How can I assist you today?'}

In [67]:
agent_executor.invoke({"input": "whats my name"})



> Entering new AgentExecutor chain...
Your name is Bob. How can I assist you today, Bob?

> Finished chain.


{'input': 'whats my name',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob. How can I assist you today, Bob?')],
 'output': 'Your name is Bob. How can I assist you today, Bob?'}

In [68]:
agent_executor.invoke({"input": "what is news about science"})



> Entering new AgentExecutor chain...

Invoking: `get_keyword_news` with `{'keyword': 'science'}`


{'status': 'ok', 'totalResults': 23651, 'articles': [{'source': {'id': 'wired', 'name': 'Wired'}, 'author': 'Dennis Mersereau', 'title': "Is the ‘Diaper’ Method in 'Twisters' a Real Solution For Massive Tornadoes? Not Exactly", 'description': 'The ‘Twister’ sequel, which brought in over $80 billion at the box office this weekend, is full of good science. But its big reveal is full of spin.', 'url': 'https://www.wired.com/story/is-twisters-diaper-method-a-real-solution-for-massive-tornadoes/', 'urlToImage': 'https://media.wired.com/photos/66984861aa3eb0e4f25763c5/191:100/w_1280,c_limit/Twisters-Review-Culture.jpg', 'publishedAt': '2024-07-22T14:13:12Z', 'content': 'OK, so the films twins are possible, but what about its flaming tornado? During one of Twisters climatic moments multiple fireballs fill the stormy skies as a violent tornado runs through an industri… [+2709 chars]'}, {'sourc

{'input': 'what is news about science',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob. How can I assist you today, Bob?'),
  HumanMessage(content='what is news about science'),
  AIMessage(content="Here are some recent news articles about science:\n\n1. Title: [Is the ‘Diaper’ Method in 'Twisters' a Real Solution For Massive Tornadoes? Not Exactly](https://www.wired.com/story/is-twisters-diaper-method-a-real-solution-for-massive-tornadoes/)\n   Source: Wired\n   Description: The ‘Twister’ sequel, which brought in over $80 billion at the box office this weekend, is full of good science. But its big reveal is full of spin.\n   Published At: 2024-07-22\n\n2. Title: [Why we might never know the truth about ultra-processed foods](https://www.bbc.com/news/articles/cpe3pppw1z7o)\n   Source: BBC News\n   Description: Experts can’t

### Create a chatbot

In [ ]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [ ]:
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [ ]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
    
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [ ]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard